第６章までで説明してきたGLMは現実のデータ解析に応用できない。

\\( \because \\)実験・調査で得られたカウントデータのバラつきは、ポワソン分布や二項分布だけではうまく説明できない。全ての個体差を説明変数で表すことは不可能。

そこで以下を用いる。

**一般化線形混合モデル(generalized linear miexed model, GLMM)**「人間が測定できない・測定しなかった個体差」を組み込んだGLM。

複数の確率分布を部品とする。
* データのバラつきは二項分布・ポアソン分布
* 個体のバラつきは正規分布

というように表す。


## 7.1 例題：GLMでは説明できないカウントデータ

* 個体数：100
* 個体iごとの調査種子数：8個
* 生存していた種子数：\\( y_i \\)個
* 葉数：\\( x_i \in \\{2,3,4,5,6\\}各葉数ごとに20個体 \\)

生存確率\\( q_i \\)を線形予測子とロジットリンク関数を組み合わせて、
$$ logit(q_i) = \beta_1 + \beta_2x_i $$

#### 補足：なぜ生存確率を線形予測子とロジットリンク関数の組み合わせとするか

二項分布のリンク関数はロジットリンク関数で表すと都合が良いため。

6.4節の要約。

ロジスティック関数

$$ q_i= logistic(z_i) = \frac{1}{1+exp(-z_i)} $$

変形すると

$$ log\frac{q_i}{1-q_i} = z_i $$

これがロジット関数(logit function)

$$ logit(q_i) = log\frac{q_i}{1-q_i} $$

これが線形予測子に等しいので、

$$  \begin{align}
\frac{q_i}{1-q_i} &= exp(線形予測子)\\
&= exp(\beta_1+\beta_2x_i+\beta_3f_i)\\
&= exp(\beta_1)exp(\beta_2x_i)exp(\beta_3f_i)
 \end{align} $$
 
\\( \frac{q_i}{1-q_i} \\)はオッズと呼ばれ、\\( \frac{生存する確率}{生存しない確率} \\)。これが
 
* \\( exp(線形予測子)\geqq 0 \\)なので推定計算に都合が良い
* 要因の効果が積で表されるのでわかりやすい

$$ p(y_i | \beta_1,\beta_2)=\binom{8}{y_i}q_i^{y_i}(1-q_i)^{8-y_i} $$

## 7.2 過分散と個体差

<code>
> d <- read.csv("data.csv")
> d4 <- d[d$x == 4,]
> table(d4$y)
0 1 2 3 4 5 6 7 8 
3 1 4 2 1 1 2 3 3 
> c(mean(d4$y), var(d4$y))
[1] 4.050000 8.365789
</code>
生存種子数\\( y_i \\)が二項分布に従うなら、その分散は8*0.5*(1-0.5)=2くらいになるはずだが、8.36になっている。

-> このデータは二項分布と呼ぶにはバラつきが大きすぎ、二項分布では説明できない。これが**過分散**の状態

### 個体差が極端な例

生存種子数が0か8かというデータでやってみる。

<code>
> d2 <-read.csv("data2.csv")
> d24 <- d2[d2$x == 4,]
> table(d24$y)
 0  8 
10 10 
> c(mean(d24$y), var(d24$y))
[1]  4.00000 16.84211
</code>

上記と同様平均種子数は4だが分散は16まで増えている。

-> 全生存数/全調査種子数という割算をしても、これだけでは減少のパターンをうまく説明できない。

#### 個体差？場所差？

## 7.3 一般化線型混合モデル

一般化線型混合モデル(GLMM)では個体差や場所差の効果を考慮した

### 